In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from bs4 import BeautifulSoup as bs
from bs4.formatter import  HTMLFormatter
from jobpostlib import (cu, datetime, hau, humanize, nu, time, wsu, speech_engine, su)
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from urllib.parse import urlparse, parse_qs, urlencode
import os
import pyperclip
import re

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 4 seconds



----

In [14]:

def get_iframe_soup(driver, url_str, verbose=False):
    wsu.driver_get_url(driver, url_str, verbose=verbose)
    iframe_css_selector = f'iframe[src^="{url_str}"]'
    wsu.wait_for(wait_count=2, verbose=False)
    assert wsu.check_presence_by_css(driver, iframe_css_selector, wait=1, verbose=verbose), f"There is no iframe on this page: {url_str}"
    
    # Switch to the iframe with a generalized CSS selector
    iframe = WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((
        By.CSS_SELECTOR, iframe_css_selector
    )))
    wsu.wait_for(wait_count=2, verbose=verbose)
    
    # Get the page soup of the iframe
    page_soup = bs(driver.page_source, 'html.parser')
    for script_soup in page_soup.find_all(name='script'):
        script_soup.decompose()
    if verbose:
        formatter_obj = HTMLFormatter(indent=4)
        html_str = page_soup.prettify(formatter=formatter_obj)
        print(html_str)
    
    return page_soup

In [15]:

def get_link_soups(driver, i=0, query_params='', verbose=False):
    
    # Get the page soup of the iframe
    search_url = 'https://uscareers-rws.icims.com/jobs/'
    iframe_url = search_url + f'search?pr={i}' + query_params
    page_soup = get_iframe_soup(driver, iframe_url, verbose=verbose)
    
    css_selector = f'a[href^="{search_url}"]'
    link_soups_list = page_soup.select(css_selector)
    
    return link_soups_list

In [16]:

driver = wsu.get_driver(verbose=True)
link_soups_list = get_link_soups(driver, i=0, query_params='&searchLocation=12781--Remote&mobile=false&width=718&height=500&bga=true&needsRedirect=false&jan1offset=-300&jun1offset=-240', verbose=False)

# Get rid of the duplicate URLs
url_strs_set = set()
pages_list = []
for link_soup in link_soups_list:
    url_str = link_soup['href']
    if re.search(r'^https://uscareers-rws\.icims\.com/jobs/\d+/', url_str):
        url_strs_set.add(url_str)
    elif re.search(r'^https://uscareers-rws\.icims\.com/jobs/search\?pr=\d+&in_iframe=1&searchLocation=12781--Remote', url_str):
        parsed_url = urlparse(url_str)
        query_params = parse_qs(parsed_url.query)
        pages_list.append(int(query_params['pr'][0]))

for i in range(1, max(pages_list)+1):
    link_soups_list = get_link_soups(driver, i, verbose=False)
    
    # Get rid of the duplicate URLs
    for link_soup in link_soups_list:
        url_str = link_soup['href']
        if re.search(r'^https://uscareers-rws\.icims\.com/jobs/\d+/', url_str):
            url_strs_set.add(url_str)
display(len(url_strs_set))

Getting the FireFox driver
platform.system() = Windows
os.name = nt
gecko_driver_path = GeckoDriverManager().install() = C:\Users\daveb\.wdm\drivers\geckodriver\win64\v0.35.0\geckodriver.exe


30

In [33]:

# Assumes this is the first time you've run this cell
files_list = []
for url_str in url_strs_set:
    page_soup = get_iframe_soup(driver, url_str, verbose=False)
    css_selector = 'h1.iCIMS_Header'
    job_title_str = page_soup.select(css_selector)[0].text.split('|')[0].strip()
    css_selector = 'div.col-xs-6:nth-child(2) > span:nth-child(2)'
    subtitles_list = page_soup.select(css_selector)
    if subtitles_list:
        job_subtitle_str = subtitles_list[0].text.strip()
        css_selector = 'div.iCIMS_JobHeaderTag:nth-child(1) > dd:nth-child(2) > span:nth-child(1)'
        job_id = page_soup.select(css_selector)[0].text.strip()
        page_title = f'{job_id} RWS {job_title_str} {job_subtitle_str}'
        file_name = su.ascii_regex.sub(' ', page_title).strip().replace(' ', '_') + '.html'
        file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
        if not os.path.isfile(file_path):
            css_selector = 'div.iCIMS_JobContent'
            article_soups = page_soup.select(css_selector)
            assert article_soups, f"You don't have any article in {file_name}"
            
            # Prettify the HTML
            html_str = article_soups[0].prettify(formatter=HTMLFormatter(indent=4))
            
            # Save the HTML to the file
            with open(file_path, 'w', encoding=nu.encoding_type) as f:
                print(f'Saving to {file_path}')
                f.write('<html>\n    <head>\n        <title>')
                f.write(page_title)
                f.write('</title>\n    </head>\n    <body>\n        <div id="jobDescriptionText">\n')
                f.write(re.sub('^', '                ', html_str, 0, re.MULTILINE))
                f.write('</div></body></html>')
            
            files_list.append(file_name)
            cu.ensure_filename(file_name, verbose=False)
            cu.set_posting_url(file_name, url_str, verbose=False)
    
speech_str = f'Fileing {len(files_list)} postings complete.'; print(speech_str); speech_engine.say(speech_str); speech_engine.runAndWait()

Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2023_1540_RWS_Search_Quality_Rater_US_MN_Remote.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2023_1543_RWS_Search_Quality_Rater_US_NC_Remote.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2023_1546_RWS_Search_Quality_Rater_US_NM_Remote.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2023_1534_RWS_Search_Quality_Rater_US_IA_Remote.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2024_2358_RWS_Professional_Services_Consultant_Lead_US_Remote_UK_Remote.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2023_1553_RWS_Search_Quality_Rater_US_WY_Remote.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2023_1542_RWS_Search_Quality_Rater_US_MT_Remote.html
Saving to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\html\2023_1538_RWS

In [34]:

t1 = time.time()
try:
    driver.close()
except Exception as e:
    print(f'{e.__class__.__name__} error: {str(e).strip()}')
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    wsu.clean_job_posting(file_path)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    assert row_div_list, f'{file_name} is missing <div id="jobDescriptionText">'
    for div_soup in row_div_list:
        child_strs_list = hau.get_navigable_children(div_soup, [])
        assert child_strs_list, f'{file_name} is missing its child strings'
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
speech_str = f'Populating {len(files_list)} out of {len(url_strs_set)} postings completed in {duration_str}'; speech_engine.say(speech_str); speech_engine.runAndWait()
files_list

['2023_1540_RWS_Search_Quality_Rater_US_MN_Remote.html', '2023_1543_RWS_Search_Quality_Rater_US_NC_Remote.html', '2023_1546_RWS_Search_Quality_Rater_US_NM_Remote.html', '2023_1534_RWS_Search_Quality_Rater_US_IA_Remote.html', '2024_2358_RWS_Professional_Services_Consultant_Lead_US_Remote_UK_Remote.html', '2023_1553_RWS_Search_Quality_Rater_US_WY_Remote.html', '2023_1542_RWS_Search_Quality_Rater_US_MT_Remote.html', '2023_1538_RWS_Search_Quality_Rater_US_MD_Remote.html', '2023_1532_RWS_Search_Quality_Rater_US_GA_Remote.html', '2023_1547_RWS_Search_Quality_Rater_US_OH_Remote.html', '2023_1550_RWS_Search_Quality_Rater_US_RI_Remote.html', '2023_1526_RWS_Search_Quality_Rater_US_TX_Remote_US_TX_Austin.html', '2024_2458_RWS_Data_Annotator_for_AI_Models_US_Remote.html', '2023_1530_RWS_Search_Quality_Rater_US_DE_Remote.html', '2023_1552_RWS_Search_Quality_Rater_US_WI_Remote.html', '2023_1535_RWS_Search_Quality_Rater_US_IL_Remote.html', '2023_1531_RWS_Search_Quality_Rater_US_FL_Remote.html', '2024


----
# Maintenance

In [ ]:

from bs4.formatter import HTMLFormatter
from bs4 import BeautifulSoup as bs

page_soup = bs(driver.page_source, 'html.parser')
for script_soup in page_soup.find_all(name='script'):
    script_soup.decompose()
formatter_obj = HTMLFormatter(indent=4)
html_str = page_soup.prettify(formatter=formatter_obj)
pyperclip.copy(html_str)